## 基本方程

本笔记本介绍了使用谱元方法解决1D弹性波方程的数值解。

\begin{equation}
\rho(x) \partial_t^2 u(x,t) = \partial_x (\mu(x) \partial_x u(x,t)) + f(x,t),
\end{equation}

解决此问题经过以下步骤：

1) 将波动方程写成弱形式。

2) 在积分求解后假设自由应力边界条件。

3) 将波场近似为一些基函数的线性组合：

\begin{equation}
u(x,t) \ \approx \ \overline{u}(x,t) \ = \ \sum_{i=1}^{n} u_i(t) \ \varphi_i(x)
\end{equation}

4) 在弱形式中，使用与$u(x, t)$中的基函数相同的基函数作为测试函数，这被称为Galerkin原理。

6) 通过考虑近似的位移场，将连续的弱形式写成线性方程组：

\begin{equation}
\mathbf{M}^T\partial_t^2 \mathbf{u} + \mathbf{K}^T\mathbf{u} = \mathbf{f}
\end{equation}

7) 使用中心差分方案进行时间外推：

\begin{equation}
\mathbf{u}(t + dt) = dt^2 (\mathbf{M}^T)^{-1}[\mathbf{f} - \mathbf{K}^T\mathbf{u}] + 2\mathbf{u} - \mathbf{u}(t-dt).
\end{equation}

其中，$\mathbf{M}$称为质量矩阵，$\mathbf{K}$是刚度矩阵。

以上的解决方案与经典有限元方法的解决方案完全相同。现在我们引入适当的基函数和积分方案，以高效地解决矩阵系统。

#### 拉格朗日多项式插值
在单元级别，我们引入拉格朗日多项式作为插值函数，并使用$\xi$作为代表单元域的空间变量：

\begin{equation}
\varphi_i \ \rightarrow \ \ell_i^{(N)} (\xi) \ := \ \prod_{j \neq i}^{N+1} \frac{\xi - \xi_j}{\xi_i-\xi_j}, \qquad   i,j = 1, 2, \dotsc , N + 1  
\end{equation}

#### 数值积分
可以通过用可以解析积分的多项式逼近替换$f(x)$来计算连续函数$f(x)$的积分。我们再次使用拉格朗日多项式作为插值函数，并获得Gauss-Lobatto-Legendre求积。在这里，GLL点用于执行积分。

\begin{equation}
\int_{-1}^1 f(x) \ dx \approx \int _{-1}^1 P_N(x) dx = \sum_{i=1}^{N+1}
 w_i f(x_i) 
\end{equation}

In [7]:
# Import all necessary libraries, this is a configuration step for the exercise.
# Please run it before the simulation code!
%matplotlib inline
import numpy as np
import matplotlib
# Show Plot in The Notebook
matplotlib.use("nbagg")
import matplotlib.pyplot as plt

from gll import gll
from lagrange1st import lagrange1st 
from ricker import ricker


In [8]:
# Initialization of setup
# ---------------------------------------------------------------
nt    = 600         # number of time steps
xmax  = 10000.        # Length of domain [m]
vs    = 2500.         # S velocity [m/s]
rho   = 2000          # Density [kg/m^3]
mu    = rho * vs**2   # Shear modulus mu
N     = 2           # Order of Lagrange polynomials
ne    = 125           # Number of elements
Tdom  = 0.1            # Dominant period of Ricker source wavelet
iplot = 5            # Plotting each iplot snapshot

# variables for elemental matrices
Me = np.zeros(N+1, dtype =  float)
Ke = np.zeros((N+1, N+1), dtype =  float)
# ----------------------------------------------------------------

# Initialization of GLL points integration weights
[xi, w] = gll(N)    # xi, N+1 coordinates [-1 1] of GLL points
                    # w Integration weights at GLL locations
# Space domain
le = xmax/ne        # Length of elements
# Vector with GLL points  
k = 0

# Initialization of physical coordinates xg in the entire space (commment added May 15, 2020)
xg = np.zeros((N*ne)+1) 
xg[k] = 0
for i in range(1,ne+1):
    for j in range(0,N):
        k = k+1
        xg[k] = (i-1)*le + .5*(xi[j+1]+1)*le

# ---------------------------------------------------------------
dxmin = min(np.diff(xg))  
eps = 0.2           # Courant value
dt = eps*dxmin/vs   # Global time step

# Mapping - Jacobian
J = le/2 
Ji = 1/J    # Inverse Jacobian

# 1st derivative of Lagrange polynomials
l1d = lagrange1st(N)   # Array with GLL as columns for each N+1 polynomial


### 2. 质量矩阵

现在我们初始化质量矩阵和刚度矩阵。一般来说，在单元级别上质量矩阵如下给出：

\begin{equation}
M_{ji}^e \ = \ w_j  \ \rho (\xi)  \ \frac{\mathrm{d}x}{\mathrm{d}\xi} \delta_{ij} \vert_ {\xi = \xi_j}
\end{equation}

我们使用GLL位置上的积分权重$w$，雅可比矩阵$J$和密度$\rho$来实现质量矩阵。然后，对质量矩阵进行全局装配，计算其逆，并显示逆质量矩阵，以直观地检查其外观。

In [9]:
# Elemental Mass matrix
# ---------------------------------------------------------------
for i in range(0, N+1):
    Me[i] = rho * w[i] * J    #stored as a vector since it's diagonal
                                
# Global Mass matrix
# --------------------------------------------------------------- 
k = -1
ng = (ne-1)*N + N + 1
M = np.zeros(2*ng) 

for i in range(1, ne+1):  
    for j in range(0, N+1): 
        k = k + 1
        if i>1:
            if j==0:
                k = k - 1
        M[k] = M[k] + Me[j]

# Inverse matrix of M 
# --------------------------------------------------------------- 
Minv = np.identity(ng)
for i in range(0,ng):
    Minv[i,i] = 1./M[i]
    
# --------------------------------------------------------------
# Display inverse mass matrix inv(M)
# --------------------------------------------------------------
plt.imshow(Minv)
plt.title('Mass Matrix $\mathbf{M}$')
plt.axis("off")
plt.tight_layout()
plt.show()     

<IPython.core.display.Javascript object>

### 3. 刚度矩阵

另一方面，在单元级别上刚度矩阵的一般形式如下：

\begin{equation}
K_{ji}^e \ = \ \sum_{k = 1}^{N+1} w_k \mu (\xi) \partial_\xi \ell_j (\xi) \partial_\xi \ell_i (\xi) \left(\frac{\mathrm{d}\xi}{\mathrm{d}x} \right)^2 \frac{\mathrm{d}x}{\mathrm{d}\xi} \vert_{\xi = \xi_k}
\end{equation}

我们使用GLL位置上的积分权重$w$，雅可比矩阵$J$和剪切应力$\mu$来实现刚度矩阵。然后，对刚度矩阵进行全局装配，并显示矩阵，以直观地检查其外观。

In [10]:
# Elemental Stiffness Matrix
# ---------------------------------------------------------------
for i in range(0, N+1):
    for j in range(0, N+1):
        for k in range(0, N+1):
            Ke[i,j] = Ke[i,j] + mu*w[k]*Ji*l1d[i,k]*l1d[j,k]
        
# Global Stiffness Matrix
# --------------------------------------------------------------- 
K = np.zeros([ng, ng])
# Values except at element boundaries 
for k in range(1,ne+1):
    i0 = (k-1)*N + 1
    j0 = i0
    for i in range(-1,N):
        for j in range(-1,N):
            K[i0+i,j0+j] = Ke[i+1,j+1]
            
# Values at element boundaries 
for k in range(2,ne+1):
    i0 = (k - 1)*N
    j0 = i0
    K[i0,j0] = Ke[0,0] + Ke[N,N]
       

# --------------------------------------------------------------
# Display stiffness matrix K
# --------------------------------------------------------------
plt.imshow(K)
plt.title('Stiffness Matrix $\mathbf{K}$')
plt.axis("off")

plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

### 4. 光谱元素解法

最后，我们利用计算得到的质量矩阵$\mathbf{M}$和刚度矩阵$\mathbf{K}$，结合有限差分外推方案，实现光谱元素解法如下：

\begin{equation}
\mathbf{u}(t + dt) = dt^2 (\mathbf{M}^T)^{-1}[\mathbf{f} - \mathbf{K}^T\mathbf{u}] + 2\mathbf{u} - \mathbf{u}(t-dt).
\end{equation}

In [11]:
# SE Solution, Time extrapolation
# ---------------------------------------------------------------

# initialize source time function and force vector f
src  = ricker(dt,Tdom)
isrc = int(np.floor(ng/2))   # Source location

# Initialization of solution vectors
u = np.zeros(ng)
uold = u
unew = u
f = u 

# --------------------------------------------------------------
# Initialize animated plot
# --------------------------------------------------------------
plt.figure(figsize=(10,6))
lines = plt.plot(xg, u, lw=1.5)
plt.title('SEM 1D Animation', size=24)
plt.xlabel(' x (m)', size=24)
plt.ylabel(' Amplitude ', size=24)
plt.rc('xtick', labelsize=20)    # fontsize of the tick labels
plt.rc('ytick', labelsize=20)    # fontsize of the tick labels
plt.ion() # set interective mode
plt.show()

# ---------------------------------------------------------------
# Time extrapolation
# ---------------------------------------------------------------
for it in range(nt): 
    # Source initialization
    f= np.zeros(ng)
    if it < len(src):
        f[isrc-1] = src[it-1] 
               
    # Time extrapolation
    unew = dt**2 * Minv @ (f - K @ u) + 2 * u - uold
    uold, u = u, unew

    # --------------------------------------   
    # Animation plot. Display solution 
    if not it % iplot:
        for l in lines:
            l.remove()
            del l
        # -------------------------------------- 
        # Display lines            
        lines = plt.plot(xg, u, color="black", lw = 1.5)
        plt.gcf().canvas.draw()       

<IPython.core.display.Javascript object>